In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from random import randint

import pytz
import datetime
from datetime import datetime, timedelta
from geopy import geocoders
from tzwhere import tzwhere
from pytz import timezone
from time import sleep

%matplotlib inline
%reload_ext autoreload
%autoreload 2

# set path
PATH = '../../data/ga_revenue/city_country_data/'
! pwd

/Users/zach-eberhart/Desktop/kaggle/notebooks/ga_revenue


# Merge All City, Country Data (w/ Country Codes)
## Read in Data

In [2]:
country_codes = pd.read_csv(PATH+'country_codes.csv', header = None)
country_codes.columns = ['country', 'code']
wc1 = pd.read_csv(PATH+'world_cities.csv', encoding = "ISO-8859-1")
wc2 = pd.read_csv(PATH+'world_cities_2.csv')
wc3 = pd.read_csv(PATH+'world_cities_3.csv')
wc4 = pd.read_csv(PATH+'world_cities_4.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
country_codes.head(3)

,country,code
0,Afghanistan,AF
1,Ã…land Islands,AX
2,Albania,AL


In [4]:
print('Shape:', wc1.shape)
wc1.head(3)

Shape: (3173958, 7)


,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,6,NaN,42.483333,1.466667
1,ad,aixirivali,Aixirivali,6,NaN,42.466667,1.500000
2,ad,aixirivall,Aixirivall,6,NaN,42.466667,1.500000


In [5]:
print('Shape:', wc2.shape)
wc2.head(3)

Shape: (23018, 4)


,name,country,subcountry,geonameid
0,les Escaldes,Andorra,Escaldes-Engordany,3040051
1,Andorra la Vella,Andorra,Andorra la Vella,3041563
2,Umm al Qaywayn,United Arab Emirates,Umm al Qaywayn,290594


In [6]:
print('Shape:', wc3.shape)
wc3.head(3)

Shape: (12893, 11)


,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Malishevë,Malisheve,42.4822,20.7458,Kosovo,XK,XKS,Malishevë,admin,NaN,1901597212
1,Prizren,Prizren,42.2139,20.7397,Kosovo,XK,XKS,Prizren,admin,NaN,1901360309
2,Zubin Potok,Zubin Potok,42.9144,20.6897,Kosovo,XK,XKS,Zubin Potok,admin,NaN,1901608808


In [7]:
print('Shape:', wc4.shape)
wc4.head(3)

Shape: (23018, 4)


,name,country,subcountry,geonameid
0,les Escaldes,Andorra,Escaldes-Engordany,3040051
1,Andorra la Vella,Andorra,Andorra la Vella,3041563
2,Umm al Qaywayn,United Arab Emirates,Umm al Qaywayn,290594


## Clean & Merge

In [8]:
# pull out the country codes from wc3 and append to the current country code list
# to catch any missing countries (or different spellings)
wc3_codes = wc3[['country', 'iso2']].drop_duplicates().reset_index(drop = True)
wc3_codes.columns = ['country', 'code']
country_codes = country_codes.append(wc3_codes).astype(str).drop_duplicates().sort_values('country').copy()

# convert all to lower case for easier merging
for c in country_codes.columns:
    country_codes[c] = country_codes[c].apply(lambda x: x.lower())
country_codes = country_codes.drop_duplicates().reset_index(drop = True).copy()

# manually add two missing countries
country_codes = country_codes.append(pd.DataFrame([{'country': 'namibia', 'code': 'na'},
                                                   {'country': 'netherlands antilles', 'code': 'an'}]
                                                 ), sort = True).reset_index(drop = True).copy()

In [9]:
print(country_codes.shape)
country_codes.head(3)

(307, 2)


,code,country
0,af,afghanistan
1,al,albania
2,dz,algeria


In [10]:
# add country names (not just codes) to wc1
wc1 = wc1[['Country', 'City']].append(wc1[['Country', 'AccentCity']].rename(columns = {'AccentCity': 'City'})).drop_duplicates().reset_index(drop = True).copy()
wc1.columns = ['code', 'city']
wc1 = wc1.merge(country_codes, on = 'code', how = 'left').dropna().reset_index(drop = True).copy()

In [23]:
wc1.head(3)

,code,city,country
0,ad,aixas,andorra
1,ad,aixirivali,andorra
2,ad,aixirivall,andorra


In [20]:
wc2 = wc2[['name', 'country']].copy()
for c in wc2.columns:
    wc2[c] = wc2[c].astype(str).apply(lambda x: x.lower())
wc2.columns = ['city', 'country']

In [26]:
wc2.head(3)

,city,country
0,les escaldes,andorra
1,andorra la vella,andorra
2,umm al qaywayn,united arab emirates


In [30]:
wc3 = wc3[['city', 'country']].append(wc3[['city_ascii', 'country']].rename(columns = {'city_ascii': 'city'})).drop_duplicates().reset_index(drop = True).copy()

for c in wc3.columns:
    wc3[c] = wc3[c].astype(str).apply(lambda x: x.lower())

In [35]:
wc3.head(3)

,city,country
0,malishevë,kosovo
1,prizren,kosovo
2,zubin potok,kosovo


In [36]:
wc4 = wc4[['name', 'country']].copy()
for c in wc4.columns:
    wc4[c] = wc4[c].astype(str).apply(lambda x: x.lower())
wc4.columns = ['city', 'country']

In [37]:
wc4.head(3)

,city,country
0,les escaldes,andorra
1,andorra la vella,andorra
2,umm al qaywayn,united arab emirates


In [39]:
select_cols = ['city', 'country']

wcm = wc1[select_cols].append(wc2[select_cols]).append(wc3[select_cols]).append(wc4[select_cols]).drop_duplicates().reset_index(drop = True)

In [57]:
wcm['city'] = wcm['city'].str.strip().copy()
wcm['country'] = wcm['country'].str.strip().copy()
wcm = wcm.drop_duplicates().reset_index(drop = True)

In [58]:
print(wcm.shape)
wcm.head(3)

(6446302, 2)


,city,country
0,aixas,andorra
1,aixirivali,andorra
2,aixirivall,andorra


## Add final list of country codes

In [88]:
missing_country_codes = {
    'aland islands': 'ax',
    'bonaire, saint eustatius and saba': 'bq',
    'cocos islands': 'cc',
    'democratic republic of the congo': 'cd',
    'republic of the congo': 'cg',
    'ivory coast': 'ci',
    'curacao': 'cw',
    'falkland islands': 'fk',
    'micronesia': 'fm',
    'north korea': 'kp',
    'palestinian territory': 'ps',
    'saint helena': 'sh',
    'east timor': 'tl',
    'vatican': 'va'
}

# append missing country codes to the full country code df
mcc = pd.DataFrame(pd.Series(missing_country_codes))
mcc['country'] = mcc.index
mcc.reset_index(drop = True, inplace = True)
mcc.columns = ['code', 'country']
country_codes = country_codes.append(mcc).reset_index(drop = True).copy()

# filter down to codes (orig used to check for missing) and merge to full df
jc = pd.DataFrame(wcm.country.drop_duplicates().reset_index(drop = True))
jc = jc.merge(country_codes, on = 'country', how = 'left').copy()

# final merge with country codes, drop dups
wcm = deepcopy(wcm.merge(jc, on = 'country', how = 'left'))
wcm = wcm.drop_duplicates().reset_index(drop = True)
wcm = deepcopy(wcm.dropna().astype(str))

## Export

In [109]:
# # write to feather
# wcm.to_feather(PATH+'world_cities_merged')